#  Mid-gestation fetal cortex dataset: Normalization and Feature Selection 


__Upstream Steps__

* QC filter on cells
* Expression filter on genes

__This notebook__

* Normalization and log10 transformation by Scanpy
* Feature selection by Triku
* Save normalized and transformed adata

# 1. Environment

## 1.1 Libraries

In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import scanpy as sc
import scanpy.external as sce
import triku as tk

#Plotting
import seaborn as sns
import matplotlib.pyplot as plt

#utils
from datetime import datetime
from scipy.sparse import csr_matrix, isspmatrix

## 1.2 Settings

* Scanpy verbosity
* Figure size    
* Result file: the file that will store the analysis results

In [ ]:
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)

In [ ]:
#results_file = '/home/..../brainomics/Dati/2_AdataNorm.h5ad'

## 1.3 Starting Computations

In [ ]:
print(datetime.now())

----

# 2. Data Load

Adata after cell and gene filtering.

In [ ]:
#adata = sc.read('/home/..../brainomics/Data/1_AdataFilt.h5ad')
#adata = sc.read('/group/brainomics/course_material/Day2/data/Ongoing/1_AdataFilt.h5ad')

In [ ]:
adata.shape

In [ ]:
isspmatrix(adata.X)

In [ ]:
print('Loaded Filtered AnnData object: number of cells', adata.n_obs)
print('Loaded Filtered AnnData object: number of genes', adata.n_vars)
 
print('Available metadata for each cell: ', adata.obs.columns)

-------

# 3. Normalize and Log Transform data: **Scanpy Normalization**

## 3.1 Store raw counts in 'counts' layer

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
print(adata.X[:, adata.var_names == 'ACTB'][:6])

## 3.2 Basic Scanpy Normalization

Some useful parameters to keep in mind from the scanpy documentation for [sc.pp.normalize_total](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.normalize_total.html)
>- `target_sum` : If None, after normalization, each observation (cell) has a total count equal to the **median of total counts for observations (cells) before normalization**.
>- `exclude_highly_expressed` : Exclude (very) highly expressed genes for the computation of the normalization factor (size factor) for each cell. **A gene is considered highly expressed, if it has more than max_fraction of the total counts in at least one cell**. The not-excluded genes will sum up to target_sum.
>- `max_fraction` : float (**default: 0.05**) If exclude_highly_expressed=True, consider cells as highly expressed that have more counts than max_fraction of the original total counts in at least one cell.

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4, exclude_highly_expressed=True)
sc.pp.log1p(adata)

## 3.3 Store normalized counts

In [ ]:
adata.layers['lognormcounts']=adata.X.copy()

In [ ]:
print(adata.layers['counts'][:, adata.var_names == 'ACTB'][:6])

In [ ]:
print(adata.layers['lognormcounts'][:, adata.var_names == 'ACTB'][:6])

In [ ]:
print(adata.X[:, adata.var_names == 'ACTB'][:6])

## Alternative workflow: normalization by **Scran**

<div class="alert alert-block alert-warning"><b>FOOD for THOUGHTS: normalize with Scran</b>

    
<nav> <b> References: </b>
<a href="https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0947-7"> Scran Paper </a> |
<a href="https://bioconductor.org/packages/release/bioc/vignettes/scran/inst/doc/scran.html">Scran R Vignette </a> |
<a href="https://github.com/theislab/single-cell-tutorial/blob/master/latest_notebook/Case-study_Mouse-intestinal-epithelium_1906.ipynb">Theis Scran Tutorial in scanpy </a> |
 </nav>
    

Normalizing cell-specific biases

Cell-specific biases are normalized using the computeSumFactors() method, which implements the deconvolution strategy for scaling normalization (A. T. Lun, Bach, and Marioni 2016). This computes size factors that are used to scale the counts in each cell. The assumption is that most genes are not differentially expressed (DE) between cells , such that any differences in expression across the majority of genes represents some technical bias that should be removed.

----

# 4. Feature selection: Triku

<nav> <b> Sources: </b>
<a href="https://academic.oup.com/gigascience/article/doi/10.1093/gigascience/giac017/6547682"> Triku Paper </a> |
<a href="https://triku.readthedocs.io/en/latest/triku-work.html"> Docs </a> |
 </nav>

The premise of triku is that, for genes with similar expression levels, the expression pattern can be categorized in three states: 

- i: the gene is expressed throughout the cells with similar expression levels **(a)**: NO useful information about specific cell types associated to that gene
- ii: the expression of the gene can be localized in a subset of cells, which can in turn be:
    - Transcriptomically different cells **(b1)** (i.e. cells that are not neighbours in the dimensionally reduced map)
    - Transcriptomically similar cells **(b2)** (neighbours): the gene is more probably biologically relevant for that population

![](https://triku.readthedocs.io/en/latest/_images/cluster_distribution.svg)

**Triku aims to select genes of case (b2)** while avoiding the selection of genes of case (a) and (b1). 

It does so by **looking at the expression in the nearest neighbours**

<div class="alert alert-block alert-success"><b>PROs </b>of this method:
    
- selects more biologically relevant genes 
- avoids selection of mitocondrial and ribosomal genes
- tends to select a lower number of HVGs aiding downstream computations
</div>

## The Algorithm 
1. **Create a neighbour graph** by selecting the k cells with the most similar transcriptome to each cell in the dataset kNN:
2. For each gene:
    1. Obtain the distribution of the the kNN counts **summing the counts of each cell and its neighbors** for each cell with positive expression in the dataset. 
    2. Simulate a **null distribution**, as above but considering k random cells instead of kNN ones. 
    3. Compare the kNN distribution with its corresponding null distribution ì
    4. **Compute the [Wasserstein distance](https://en.wikipedia.org/wiki/Wasserstein_metric)** between both distributions
3. Normalize Wasserstein distances
4. Select the features with highest Wasserstein distance. 

![](https://triku.readthedocs.io/en/latest/_images/knn_scheme.svg)

## 4.1 Compute neighbors

In [ ]:
sc.pp.pca(adata, use_highly_variable=False) #specify in case you want to try different HVG selection methods 

In [ ]:
sc.pp.neighbors(adata, metric='cosine', n_neighbors=int(0.5 * len(adata) ** 0.5)) 

## 4.2 Identify HVG

In [ ]:
tk.tl.triku(adata, use_raw=False)

In [ ]:
Top20Triku = adata.var.sort_values(by=['triku_distance'], ascending=False).head(20).index
Top20Triku

In [ ]:
print('Number of Higly Variable Genes', len(adata.var_names[adata.var['highly_variable'] == True]))


<div class="alert alert-block alert-warning"> <b>FOOD for THOUGHTS: Batch effect could influence HVG selection. </b> 
    
You can consider to correct (e.g. with Harmony) the neighbors used to compute the HVGs. For this dataset we did not notice a big difference between the two procedures.

----

<div class="alert alert-block alert-warning"><b>FOOD for THOUGHTS: Alternative HVG approach using scanpy function</b>


    
 > sc.pp.highly_variable_genes(adata) # n_top_genes=2000,  #batch_key='batch'
    
 > sc.pl.highly_variable_genes(adata)
 
 > print('Number of Higly Variable Genes', len(adata.var_names[adata.var['highly_variable'] == True]))
 
> adata.var['scanpy_highly_variable'] = adata.var['highly_variable'] 

> adata.var['highly_variable'] = adata.var['triku_highly_variable'] 

----

# 5. Save

## 5.1 Save AData

In [ ]:
del adata.uns['triku_params']
adata.write(results_file)

## 5.2 Timestamp finished computations 

In [ ]:
print(datetime.now())

## 5.3 Save python and html versions

In [ ]:
#nb_fname = ipynbname.name()
nb_fname='2_Norm_HVG'

In [ ]:
%%bash -s "$nb_fname"
jupyter nbconvert "$1".ipynb --to="python"
jupyter nbconvert "$1".ipynb --to="html"